In [1]:
import torch
import torch.nn as nn
import numpy as np
import sys
import os
import random
import matplotlib.pyplot as plt

src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)
    
from utils import MIMONetDataset, DeepONetDataset, ChannelScaler
from mimonet import MIMONet

In [2]:
# check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [3]:
# set working directory
working_dir = "/projects/bcnx/kazumak2/MIMONet/HeatExchanger"
data_dir = os.path.join(working_dir, "data")

## load datasets

### Load sharing parameters/dataset

In [4]:
# trunk dataset
trunk_input = np.load(os.path.join(data_dir, "share/trunk.npz"))['trunk']

### Training data

In [34]:
# branch input dataset
branch = np.load(os.path.join(data_dir, "branch.npz"))

branch1 = branch['branch1']
branch2 = branch['branch2']

print("Branch1 shape:", branch1.shape)
print("Branch2 shape:", branch2.shape)

# split the dataset into training and testing sets
train_size = int(0.8 * len(branch1))
test_size = len(branch1) - train_size
train_branch1, test_branch1 = branch1[:train_size], branch1[train_size:]
train_branch2, test_branch2 = branch2[:train_size], branch2[train_size:]

Branch1 shape: (1546, 2)
Branch2 shape: (1546, 100)


In [35]:
# output channels
# 0: turb-kinetic-energy
# 1: pressure
# 2: temperature
# 3: z-velocity
# 4: y-velocity
# 5: x-velocity
# 6: velocity-magnitude

# target dataset
target = np.load(os.path.join(data_dir, "target.npy"))

print("Target shape:", target.shape)

# split the target dataset into training and testing sets
train_target = target[:train_size]
test_target = target[train_size:]

print("Train target shape:", train_target.shape)
print("Test target shape:", test_target.shape)

Target shape: (1546, 3977, 7)
Train target shape: (1236, 3977, 7)
Test target shape: (310, 3977, 7)


In [6]:
# (# train samples, 2) 
# get the mean and standard deviation of each channel
mean_branch1 = np.mean(train_branch1, axis=0)
std_branch1 = np.std(train_branch1, axis=0)

print("Mean of branch1:", mean_branch1)
print("Std of branch1:", std_branch1)

# (# train samples, 100)
mean_branch2 = np.mean(train_branch2)
std_branch2 = np.std(train_branch2)

print("Mean of branch2:", mean_branch2)
print("Std of branch2:", std_branch2)

Mean of branch1: [  4.51454429 292.42944177]
Std of branch1: [ 0.2615285  17.03323994]
Mean of branch2: 12587.66968713018
Std of branch2: 6302.709013835411


In [7]:
# normalize the branch data using the mean and std
train_branch_1 = (train_branch1 - mean_branch1) / std_branch1
test_branch_1 = (test_branch1 - mean_branch1) / std_branch1
train_branch_2 = (train_branch2 - mean_branch2) / std_branch2
test_branch_2 = (test_branch2 - mean_branch2) / std_branch2

# print the shapes of the normalized data
print("Shape of normalized train_branch1:", train_branch_1.shape)
print("Shape of normalized test_branch1:", test_branch_1.shape)
print("Shape of normalized train_branch2:", train_branch_2.shape)
print("Shape of normalized test_branch2:", test_branch_2.shape)

Shape of normalized train_branch1: (1236, 2)
Shape of normalized test_branch1: (310, 2)
Shape of normalized train_branch2: (1236, 100)
Shape of normalized test_branch2: (310, 100)


### Scaling the target data

In [8]:
# scaling the target data
'''  
note: reverse the scaling for the target data
train_target = scaler.inverse_transform(train_target_scaled)
test_target = scaler.inverse_transform(test_target_scaled)
'''
scaler = ChannelScaler(method='minmax', feature_range=(-1, 1))
scaler.fit(train_target)
train_target_scaled = scaler.transform(train_target)
test_target_scaled = scaler.transform(test_target)

print("Shape of scaled train_target:", train_target_scaled.shape)
print("Shape of scaled test_target:", test_target_scaled.shape)

Shape of scaled train_target: (1236, 3977, 7)
Shape of scaled test_target: (310, 3977, 7)


## Torch Dataset and DataLoader

In [9]:
# test dataset and dataloader
test_dataset = MIMONetDataset(
    [test_branch_1, test_branch_2],  # branch_data_list
    trunk_input,                     # trunk_data
    test_target_scaled               # target_data
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # set to 1 for testing
    shuffle=False,
    num_workers=0
)

In [10]:
train_dataset = MIMONetDataset(
    [train_branch_1, train_branch_2],  # branch_data_list
    trunk_input,                       # trunk_data
    train_target_scaled                # target_data
)

## MIMONet Model

In [11]:
# Architecture parameters
dim = 256
branch_input_dim1 = 2
branch_input_dim2 = 100
trunk_input_dim = 2

# Define the model arguments for orig_MIMONet
model_args = {
    'branch_arch_list': [
        [branch_input_dim1, 512, 512, 512, dim],
        [branch_input_dim2, 512, 512, 512, dim]
    ],
    'trunk_arch': [trunk_input_dim, 256, 256, 256, dim],
    'num_outputs': 7,
    'activation_fn': nn.ReLU,
    'merge_type': 'mul',
}

In [12]:
from training import train_model

In [13]:
# scheduler parameters
scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau
scheduler_args={'mode': 'min', 'factor': 0.5, 'patience': 10,}

In [14]:
def run():
    train_model(
        model_fn=MIMONet,
        model_args=model_args,
        optimizer_fn=torch.optim.Adam,
        optimizer_args={'lr': 1e-3, 'weight_decay': 1e-6},
        scheduler_fn=scheduler_fn,
        scheduler_args=scheduler_args,
        dataset=train_dataset,
        device=device,
        num_epochs=3,
        batch_size=4,
        criterion=nn.MSELoss(),
        patience=500,
        k_fold=None,
        multi_gpu=False,
        working_dir="/projects/bcnx/kazumak2/MIMONet/HeatExchanger/",
    )

# start the training
run()

Epoch 1, LR: 1.00e-03, Train Loss: 0.100594, Val Loss: 0.096711
Epoch 2, LR: 1.00e-03, Train Loss: 0.094217, Val Loss: 0.089276
Epoch 3, LR: 1.00e-03, Train Loss: 0.077679, Val Loss: 0.060111
Best model saved at: /projects/bcnx/kazumak2/MIMONet/HeatExchanger/checkpoints/best_model.pt


### Evaluation with Test Dataset

In [17]:
model = MIMONet(**model_args).to(device)
# Load the trained model
model_path = os.path.join(working_dir, "checkpoints/best_model.pt")

if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
    print("Model loaded successfully from", model_path)
else:
    print("Model file not found at", model_path)
    sys.exit(1)
    
# Evaluate the model on the test set
model.eval()

Model loaded successfully from /projects/bcnx/kazumak2/MIMONet/HeatExchanger/checkpoints/best_model.pt


MIMONet(
  (branch_nets): ModuleList(
    (0): FCN(
      (network): Sequential(
        (0): Linear(in_features=2, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): ReLU()
        (4): Linear(in_features=512, out_features=512, bias=True)
        (5): ReLU()
        (6): Linear(in_features=512, out_features=256, bias=True)
      )
    )
    (1): FCN(
      (network): Sequential(
        (0): Linear(in_features=100, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): ReLU()
        (4): Linear(in_features=512, out_features=512, bias=True)
        (5): ReLU()
        (6): Linear(in_features=512, out_features=256, bias=True)
      )
    )
  )
  (trunk_net): FCN(
    (network): Sequential(
      (0): Linear(in_features=2, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (

In [18]:
# feed the test loader to the model (and save predictions and targets)
predictions = []
targets = []
with torch.no_grad():
    for batch in test_loader:
        branch_data, trunk_data, target_data = batch
        branch_data = [b.to(device) for b in branch_data]
        trunk_data = trunk_data.to(device)
        target_data = target_data.to(device)

        output = model(branch_data, trunk_data)
        predictions.append(output.cpu().numpy())
        targets.append(target_data.cpu().numpy())
# Convert predictions and targets to numpy arrays
predictions = np.concatenate(predictions, axis=0)
targets = np.concatenate(targets, axis=0)
# Reverse the scaling for the target data
predictions = scaler.inverse_transform(predictions)
targets = scaler.inverse_transform(targets)

In [32]:
#fig, ax = plt.subplots(1, 2, figsize=(12, 6))
#
#y = trunk_input[:, 0]
#z = trunk_input[:, 1]
#
## set the output channel
#ch_id = 2
#
#
#ax[0].scatter(y, z, c=targets[0, :, ch_id], cmap='viridis', s=1)
## add colorbar
#cbar = plt.colorbar(ax[0].collections[0], ax=ax[0])
#
#ax[1].scatter(y, z, c=test_target[0, :, ch_id], cmap='viridis', s=1)
#cbar = plt.colorbar(ax[1].collections[0], ax=ax[1])
#
#plt.show()